In [1]:
import numpy as np
import cplex
import random

In [2]:
class Subproblem_LP():
    def __init__(self):
        self.n_dim = (n_stage - 1) * n_job * n_job
        self.n_constraints = 2 * (n_stage - 1) * n_job
        self.variable_names = ['x'+ str(i) for i in range(self.n_dim)]
        self.constraints_rows = self.set_constraints_byrows()
        self.opt_solution = np.random.randint(0,2,size = self.n_dim) 
        self.rhs = [1] * self.n_constraints
        self.senses = 'E'* self.n_constraints

    def set_job_to_position(self):
        job_to_position_onlyone = []
        for i in range((n_stage - 1) * n_job):
            job_to_position_onlyone.append([self.variable_names[i * n_job : (i + 1) * n_job], [1] * n_job])
        return job_to_position_onlyone
    
    def set_position_to_job(self):
        position_to_job_onlyone = []
        for j in range(n_stage - 1):
            for i in range(n_job):
                position_to_job_onlyone.append([self.variable_names[j * (n_job**2) + i: (j + 1) * (n_job**2): n_job], [1] * n_job])
        return position_to_job_onlyone
    
    def set_constraints_byrows(self):
        job_to_position_onlyone = self.set_job_to_position()
        position_to_job_onlyone = self.set_position_to_job()
        return job_to_position_onlyone + position_to_job_onlyone
    
    def compute_obj(self, tval, lamd):
        x_coeff_f4 = self.compute_f4_x(lamd)
        x_coeff_F6 = self.compute_F6_x(tval, lamd)
        x_coeff = [0 for i in range((n_stage - 1) * n_job * n_job)]
        for i in range(n_job):
            for j in range(n_stage-1):
                for r in range(n_batch * pcast):
                    for k in range(n_machine_perstage):
                        for i1 in range(n_job):
                            t2 = i * n_job + j * n_job * n_job + k * n_batch * pcast + r
                            x_coeff[t2] = x_coeff_f4[t2] + x_coeff_F6[t2]
        self.obj = x_coeff
        return self.obj
    
    def compute_F6_x(self, tval, lamd):
        x_coeff = [0 for i in range((n_stage - 1) * n_job * n_job)]
        for i in range(n_job):
            for j in range(n_stage-1):
                t1 = i + j * n_job
                for r in range(n_batch * pcast):
                    for k in range(n_machine_perstage):
                        t2 = i * n_job + j * n_job * n_job + k * n_batch * pcast + r
                        for i1 in range(n_job):
                            if r == n_batch * pcast - 1:
                                t3 = i1 * n_job + i * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r - 1
                                t4 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r - 1 
                                t5 = i1 + j * n_job
                                x_coeff[t2] += lamd[t3] * ((0.5 * ((self.opt_solution[t2] + self.opt_solution[t4])**2 - self.opt_solution[t2] - self.opt_solution[t4]) + tval[t1] -  tval[t5]) * (self.opt_solution[t2] + self.opt_solution[t4] - 0.5) - 0.5 * (2 * processtime[t5] + 1) * (self.opt_solution[t4] - self.opt_solution[t2]))           
                            elif r == 0:
                                t3 = i * n_job + i1 * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r
                                t4 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r + 1
                                t5 = i1 + j * n_job
                                x_coeff[t2] += lamd[t3] * ((0.5 * ((self.opt_solution[t2] + self.opt_solution[t4])**2 - self.opt_solution[t2] - self.opt_solution[t4]) + tval[t5] -  tval[t1]) * (self.opt_solution[t2] + self.opt_solution[t4] - 0.5) + 0.5 * (2 * processtime[t1] + 1) * (self.opt_solution[t2] - self.opt_solution[t4]))           
                            else:
                                t3 = i * n_job + i1 * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r
                                t4 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r + 1
                                t5 = i1 + j * n_job
                                t6 = i1 * n_job + i * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r - 1
                                t7 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r - 1
                                x_coeff[t2] += lamd[t3] * ((0.5 * ((self.opt_solution[t2] + self.opt_solution[t4])**2 - self.opt_solution[t2] - self.opt_solution[t4]) + tval[t5] -  tval[t1]) * (self.opt_solution[t2] + self.opt_solution[t4] - 0.5) + 0.5 * (2 * processtime[t1] + 1) * (self.opt_solution[t2] - self.opt_solution[t4]))
                                x_coeff[t2] += lamd[t6] * ((0.5 * ((self.opt_solution[t2] + self.opt_solution[t7])**2 - self.opt_solution[t2] - self.opt_solution[t7]) + tval[t1] -  tval[t5]) * (self.opt_solution[t2] + self.opt_solution[t7] - 0.5) - 0.5 * (2 * processtime[t5] + 1) * (self.opt_solution[t7] - self.opt_solution[t2]))           
        return x_coeff
    
    def compute_f4_x(self, lamd):
        x_coeff = [0 for i in range((n_stage - 1) * n_job * n_job)]
        for i in range(n_job):
            for j in range(n_stage - 1):
                t1 = i + j * n_job
                for r in range(n_batch * pcast):
                    for k in range(n_machine_perstage):
                        t2 = i * n_job + j * n_job * n_job + k * n_batch * pcast + r
                        for i1 in range(n_job):
                            if r == n_batch * pcast - 1:
                                t3 = i * n_job + i1 * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + n_batch * pcast - 1
                                x_coeff[t2] += 0.25 * (2 * processtime[t1] + 1) * lamd[t3]
                            elif r == 0:
                                t3 = i * n_job + i1 * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast
                                x_coeff[t2] += 0.25 * (2 * processtime[t1] + 1) * lamd[t3]
                            else:
                                t3 = i * n_job + i1 * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r
                                t4 = i1 * n_job + i * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r + 1
                                x_coeff[t2] += 0.25 * (2 * processtime[t1] + 1) * (lamd[t3] + lamd[t4])
        return x_coeff

    def set_method(self, method):
        alg = self.subproblem_cplex.parameters.lpmethod.values

        if method == "o":  
            self.subproblem_cplex.parameters.lpmethod.set(alg.auto)
        elif method == "p":
            self.subproblem_cplex.parameters.lpmethod.set(alg.primal)
        elif method == "d":
            self.subproblem_cplex.parameters.lpmethod.set(alg.dual)
        elif method == "b":
            self.subproblem_cplex.parameters.lpmethod.set(alg.barrier)
            self.subproblem_cplex.parameters.barrier.crossover.set(self.Subproblem_LP_TU.parameters.parameters.barrier.crossover.values.none)
        elif method == "h":
            self.subproblem_cplex.parameters.lpmethod.set(alg.barrier)
        elif method == "s":
            self.subproblem_cplex.parameters.lpmethod.set(alg.sifting)
        elif method == "c":
            self.subproblem_cplex.parameters.lpmethod.set(alg.concurrent)
        else:
            raise ValueError('method must be one of "o", "p", "d", "b", "h", "s" or "c"')
        
    def solve(self, method = 'o'):
        self.subproblem_cplex = cplex.Cplex()
        self.subproblem_cplex.objective.set_sense(self.subproblem_cplex.objective.sense.maximize)
        ub = [1] * self.n_dim
    
        self.subproblem_cplex.variables.add(obj = self.obj, ub = ub, names = self.variable_names)    
    
        assert(self.n_constraints == len(self.constraints_rows))
        row_names = ['c'+ str(i) for i in range(self.n_constraints)]
        print("the dimension of x:", self.n_dim, "the number of constraints:", self.n_constraints)
        self.subproblem_cplex.linear_constraints.add(lin_expr = self.constraints_rows, senses = self.senses, rhs = self.rhs, names = row_names)
        self.set_method(method)
        self.subproblem_cplex.solve()
        self.get_solvable_status()
        self.get_optvalues()
        
    def write_model(self):
        self.subproblem_cplex.write('Subproblem_LP.lp')
        
    def get_optvalues(self):
        for i, x in enumerate(self.subproblem_cplex.solution.get_values()):
            if i < self.n_dim:
                self.opt_solution[i] = x
                
    def get_solvable_status(self):
        status = self.subproblem_cplex.solution.get_status()
        if status == self.subproblem_cplex.solution.status.unbounded:
            print("Model is unbounded")
        if status == self.subproblem_cplex.solution.status.infeasible:
            print("Model is infeasible")
        if status == self.subproblem_cplex.solution.status.infeasible_or_unbounded:
            print("Model is infeasible or unbounded")
            
    def output_results(self):
        with open('Subproblem_LP_result.txt', 'w') as f:
            for i in range(n_job * n_stage):
                f.write(str(self.opt_solution[i * n_job:(i + 1) * n_job]))
                f.write('\n')

In [3]:
np.random.seed(0)
random.seed(0)
n_stage, pcast, n_batch, n_machine_perstage = 3, 5, 3, 3
#n_stage, pcast, n_batch, n_machine_perstage = 3, 1, 3, 2
stage_name = ['steelmaking', 'refining', 'casting']
n_machine = dict(zip(stage_name, [n_machine_perstage for i in range(n_stage)]))
n_job = pcast * n_batch * n_machine['steelmaking']
assert(len(stage_name) == n_stage)

print("Number of stage:", n_stage)
print("Cast number of per machine:", pcast)
print("Number of job per cast:", n_batch)
print('Number of machine:', n_machine)
print('Number of job:', n_job)

n_position = dict(zip(stage_name[0:n_stage - 1], [int(n_job / n_machine['steelmaking'])] * (n_stage - 1)))
print('n_position = ', n_position)
assert(len(n_machine) == n_stage)
assert(len(n_position) == n_stage - 1)

processtime = [random.randint(36, 50) for i in range(n_job * n_stage)]
transittime = [random.randint(3, 6) for i in range(n_stage - 1)]
su = 80
duedate = [0 for i in range(pcast * n_machine['casting'])]

penalty_coeff = 10
d_coeff = {'d1': 10, 'd2': 110}
coeff = [10 + 20 * i for i in range(n_stage - 1)]
print(processtime)
print(transittime)

Number of stage: 3
Cast number of per machine: 5
Number of job per cast: 3
Number of machine: {'steelmaking': 3, 'refining': 3, 'casting': 3}
Number of job: 45
n_position =  {'steelmaking': 15, 'refining': 15}
[49, 42, 48, 50, 42, 36, 40, 44, 43, 42, 50, 48, 49, 40, 43, 41, 45, 50, 50, 39, 44, 38, 40, 38, 48, 37, 45, 48, 40, 50, 44, 47, 48, 45, 50, 38, 40, 37, 47, 37, 50, 49, 46, 41, 43, 44, 37, 41, 42, 41, 45, 46, 50, 39, 44, 43, 43, 49, 44, 40, 36, 48, 50, 44, 50, 36, 37, 47, 49, 42, 47, 49, 48, 46, 46, 36, 45, 43, 49, 49, 41, 39, 47, 41, 47, 49, 37, 39, 50, 45, 39, 39, 48, 38, 48, 44, 43, 37, 37, 41, 50, 44, 50, 43, 37, 40, 44, 40, 47, 37, 44, 41, 49, 50, 44, 39, 48, 45, 44, 45, 40, 43, 37, 45, 48, 42, 41, 45, 39, 40, 38, 39, 49, 38, 36]
[5, 6]


In [4]:
subproblem_lp = Subproblem_LP() 
job_to_position_onlyone = subproblem_lp.set_job_to_position()
print(len(job_to_position_onlyone))
print(job_to_position_onlyone)

90
[[['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20', 'x21', 'x22', 'x23', 'x24', 'x25', 'x26', 'x27', 'x28', 'x29', 'x30', 'x31', 'x32', 'x33', 'x34', 'x35', 'x36', 'x37', 'x38', 'x39', 'x40', 'x41', 'x42', 'x43', 'x44'], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], [['x45', 'x46', 'x47', 'x48', 'x49', 'x50', 'x51', 'x52', 'x53', 'x54', 'x55', 'x56', 'x57', 'x58', 'x59', 'x60', 'x61', 'x62', 'x63', 'x64', 'x65', 'x66', 'x67', 'x68', 'x69', 'x70', 'x71', 'x72', 'x73', 'x74', 'x75', 'x76', 'x77', 'x78', 'x79', 'x80', 'x81', 'x82', 'x83', 'x84', 'x85', 'x86', 'x87', 'x88', 'x89'], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], [['x90', 'x91', 'x92', 'x93', 'x94', 'x95', 'x96', 'x97', 'x98', 'x99', 'x100', 'x101', 'x102'

In [5]:
position_to_job_onlyone = subproblem_lp.set_position_to_job()
print(len(position_to_job_onlyone))
print(position_to_job_onlyone)

90
[[['x0', 'x45', 'x90', 'x135', 'x180', 'x225', 'x270', 'x315', 'x360', 'x405', 'x450', 'x495', 'x540', 'x585', 'x630', 'x675', 'x720', 'x765', 'x810', 'x855', 'x900', 'x945', 'x990', 'x1035', 'x1080', 'x1125', 'x1170', 'x1215', 'x1260', 'x1305', 'x1350', 'x1395', 'x1440', 'x1485', 'x1530', 'x1575', 'x1620', 'x1665', 'x1710', 'x1755', 'x1800', 'x1845', 'x1890', 'x1935', 'x1980'], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], [['x1', 'x46', 'x91', 'x136', 'x181', 'x226', 'x271', 'x316', 'x361', 'x406', 'x451', 'x496', 'x541', 'x586', 'x631', 'x676', 'x721', 'x766', 'x811', 'x856', 'x901', 'x946', 'x991', 'x1036', 'x1081', 'x1126', 'x1171', 'x1216', 'x1261', 'x1306', 'x1351', 'x1396', 'x1441', 'x1486', 'x1531', 'x1576', 'x1621', 'x1666', 'x1711', 'x1756', 'x1801', 'x1846', 'x1891', 'x1936', 'x1981'], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [6]:
set_constraints_byrows = subproblem_lp.set_constraints_byrows()
print(len(set_constraints_byrows))
print(set_constraints_byrows)

180
[[['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20', 'x21', 'x22', 'x23', 'x24', 'x25', 'x26', 'x27', 'x28', 'x29', 'x30', 'x31', 'x32', 'x33', 'x34', 'x35', 'x36', 'x37', 'x38', 'x39', 'x40', 'x41', 'x42', 'x43', 'x44'], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], [['x45', 'x46', 'x47', 'x48', 'x49', 'x50', 'x51', 'x52', 'x53', 'x54', 'x55', 'x56', 'x57', 'x58', 'x59', 'x60', 'x61', 'x62', 'x63', 'x64', 'x65', 'x66', 'x67', 'x68', 'x69', 'x70', 'x71', 'x72', 'x73', 'x74', 'x75', 'x76', 'x77', 'x78', 'x79', 'x80', 'x81', 'x82', 'x83', 'x84', 'x85', 'x86', 'x87', 'x88', 'x89'], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], [['x90', 'x91', 'x92', 'x93', 'x94', 'x95', 'x96', 'x97', 'x98', 'x99', 'x100', 'x101', 'x102

In [7]:
np.random.seed(0)
tval = np.random.randint(0, 2, size = n_stage * n_job + 2 * pcast * n_machine['steelmaking'])
print(tval)
lamd = np.zeros(n_job**3 * (n_stage - 1))
x_coeff = subproblem_lp.compute_F6_x(tval, lamd)
print(x_coeff)

[0 1 1 0 1 1 1 1 1 1 1 0 0 1 0 0 0 0 0 1 0 1 1 0 0 1 1 1 1 0 1 0 1 0 1 1 0
 1 1 0 0 1 0 1 1 1 1 1 0 1 0 1 1 1 1 0 1 0 0 1 1 0 1 0 1 0 0 0 0 0 1 1 0 0
 0 1 1 0 1 0 0 1 0 1 1 1 1 1 1 0 1 1 0 0 1 0 0 1 1 0 1 0 0 1 0 0 0 1 1 0 1
 0 0 0 0 0 1 0 1 0 1 1 1 1 1 0 1 1 1 1 0 1 1 0 0 1 0 0 0 0 1 1 0 0 1 0 1 1
 1 1 0 0 0 1 0 1 1 1 0 1 0 0 1 0 1]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [8]:
x_coeff = subproblem_lp.compute_f4_x(lamd)
print(x_coeff)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [9]:
x_coeff = subproblem_lp.compute_obj(tval, lamd)
print(x_coeff)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [10]:
subproblem_lp.solve()
subproblem_lp.write_model()
subproblem_lp.output_results()

the dimension of x: 4050 the number of constraints: 180
CPXPARAM_Read_DataCheck                          1
Parallel mode: deterministic, using up to 8 threads for concurrent optimization.
Tried aggregator 1 time.
No LP presolve or aggregator reductions.
Presolve time = 0.02 sec. (1.14 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Dual objective     =             0.000000
Iteration:   100   Dual objective     =             0.000000
Perturbation started.
Iteration:   101   Dual objective     =             0.000000
Iteration:   175   Dual objective     =            -0.000002
Iteration:   250   Dual objective     =            -0.000003
Removing perturbation.

Dual simplex solved model.



In [27]:
class Subproblem_QP():
    def __init__(self):
        self.n_dim = n_job * n_stage + 2 * pcast * n_machine['steelmaking']
        self.variable_names = ['t' + str(i)  for i in range(self.n_dim)]
        self.duedate = self.compute_duedate()
        self.constraints_rows, self.rhs, self.n_per_constraints = self.set_constraints_byrows()
        self.senses = self.set_senses()
        self.opt_solution = [random.randint(0,1) for i in range(n_stage * n_job)] + 2 * pcast * n_machine['steelmaking'] * [0]
        self.n_constraints = len(self.constraints_rows)
        self.cost_function_iterations = []
        
    def set_preceding_transfer_constraints(self):
        preceding_transfer_constraints = []
        for i in range(n_job * (n_stage - 1)):
            preceding_transfer_constraints.append([[self.variable_names[i], self.variable_names[i + n_job]], [1, -1]])

        rhs_transittime, rhs_processtime = [], processtime[0: n_job * (n_stage - 1)]
        for i in range(len(transittime)):
            rhs_transittime += [transittime[i]] * n_job
        rhs = [ - processtime[i] - rhs_transittime[i] for i in range(n_job * (n_stage - 1))]
        return preceding_transfer_constraints, rhs, rhs_processtime, rhs_transittime

    def set_process_continuous_constraints(self):
        process_continuous, adjacent_casts = [], []
        rhs_continuous_processtime, rhs_adjacent_casts = [], []
        assert(n_job * (n_stage - 1) % n_batch == 0)
        for i in range(n_job * (n_stage - 1), n_job * n_stage - 1):
            if (i + 1) % n_batch != 0:
                process_continuous.append([[self.variable_names[i], self.variable_names[i + 1]], [1, -1]])
                rhs_continuous_processtime.append(processtime[i])
            else:
                if (i + 1) % (n_batch * pcast) != 0:
                    adjacent_casts.append([[self.variable_names[i], self.variable_names[i + 1]], [1, -1]])
                    rhs_adjacent_casts.append(- processtime[i] - su)
        return process_continuous, adjacent_casts, rhs_continuous_processtime, rhs_adjacent_casts 

    def set_additional_constraints(self):
        additional_constraints = []
        index = n_job * n_stage
        index_start = pcast * n_machine['casting']
        assert(n_job * (n_stage - 1) % n_batch == 0 and (index % n_batch == 0))
        batch = 0
        for i in range(n_job * (n_stage - 1), n_job * n_stage - 1):
            if i % n_batch == 0:
                additional_constraints.append([[self.variable_names[i], self.variable_names[index + batch], self.variable_names[index + batch + index_start]], [1, -1, 1]])
                batch += 1
        return additional_constraints
    
    def set_constraints_byrows(self):
        preceding_transfer_constraints, rhs_preceding_transfer, _, _ = self.set_preceding_transfer_constraints()
        process_continuous_constraints, adjacent_casts_constraints, rhs_continuous_processtime, rhs_adjacent_casts = self.set_process_continuous_constraints()
        additional_constraints = self.set_additional_constraints()
    
        rows = preceding_transfer_constraints + process_continuous_constraints + adjacent_casts_constraints + additional_constraints
        
        n_per_constraints = [len(preceding_transfer_constraints), len(process_continuous_constraints), len(adjacent_casts_constraints), len(additional_constraints)]
        
        rhs = rhs_preceding_transfer + rhs_continuous_processtime + rhs_adjacent_casts + self.duedate
        return rows, rhs, n_per_constraints
    
    def set_senses(self):
        s, senses = 'LELE', ''
        for i in range(len(self.n_per_constraints)):
            senses += s[i] * self.n_per_constraints[i]
        return senses
    
    def compute_duedate(self):
        duedate = [0 for i in range(pcast * n_machine['casting'])]
        duedate_start = [0 for i in range(n_machine['casting'])]
        n_cast = pcast * n_machine['casting']
        for i in range(n_machine['casting']):
            index = 0
            for j in range(n_stage - 1):
                duedate_start[i] += processtime[i * n_cast + index * n_job] + transittime[j]
                index += 1
        print(duedate_start)
    
        for i in range(n_machine['casting']):
            duedate[i * pcast] = duedate_start[i]
            index_start = n_job * (n_stage - 1) + i * n_cast
            for j in range(1, pcast):
                duedate[i * pcast + j] = duedate[i * pcast + j - 1] + sum(processtime[index_start: index_start + n_batch]) + su
                index_start = index_start + n_batch
        return duedate
    
    def compute_obj_wait_time(self):
        obj_wait_time = np.zeros((n_stage, n_job))
        for i in range(n_stage - 1):
            for j in range(n_job):
                obj_wait_time[i + 1][j], obj_wait_time[i][j] = obj_wait_time[i + 1][j] + coeff[i], obj_wait_time[i][j] - coeff[i]
        obj_wait_time = obj_wait_time.reshape(1, n_stage * n_job)
        return np.hstack((obj_wait_time, np.zeros((1, self.n_dim - n_stage * n_job)))).tolist()[0]

    def compute_obj_casting(self):
        obj_casting = np.hstack((d_coeff['d1'] *  np.ones((1, pcast * n_machine['casting'])), d_coeff['d2'] * np.ones((1, pcast * n_machine['casting']))))
        obj_casting = np.hstack((np.zeros((1, n_job * n_stage)), obj_casting))
        return obj_casting.tolist()[0]
    
    def compute_obj_linear(self, xval, lamd):
        obj_linear_wait_time = self.compute_obj_wait_time()
        obj_linear_casting = self.compute_obj_casting()
        obj_linear_F6_t = self.compute_F6_t(xval, lamd)
        self.obj_linear = [- obj_linear_wait_time[i] - obj_linear_casting[i] - obj_linear_F6_t[i] for i in range(self.n_dim)]
        return self.obj_linear
    
    def compute_F6_t(self, xval, lamd):
        t_coeff = [0 for i in range((n_stage - 1) * n_job)]
        for i in range(n_job):
            for j in range(n_stage - 1):
                t1 = i + j * n_job
                for r in range(n_batch * pcast):
                    for k in range(n_machine_perstage):
                        t2 = i * n_job + j * n_job * n_job + k * n_batch * pcast + r
                        for i1 in range(n_job):
                            if r == n_batch * pcast - 1:
                                t3 = i1 * n_job + i * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r - 1
                                t4 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r - 1 
                                t5 = i1 + j * n_job
                                t_coeff[t1] += lamd[t3] * (0.5 * ((xval[t2] + xval[t4])**2 - xval[t2] - xval[t4]) + self.opt_solution[t1] -  self.opt_solution[t5])          
                            elif r == 0:
                                t3 = i * n_job + i1 * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r
                                t4 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r + 1
                                t5 = i1 + j * n_job
                                t_coeff[t1] -= lamd[t3] * (0.5 * ((xval[t2] + xval[t4])**2 - xval[t2] - xval[t4]) + self.opt_solution[t5] -  self.opt_solution[t1])
                            else:
                                t3 = i * n_job + i1 * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r
                                t4 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r + 1
                                t5 = i1 + j * n_job
                                t6 = i1 * n_job + i * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r - 1
                                t7 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r - 1
                                t_coeff[t1] -= lamd[t3] * (0.5 * ((xval[t2] + xval[t4])**2 - xval[t2] - xval[t4]) + self.opt_solution[t5] -  self.opt_solution[t1]) 
                                t_coeff[t1] += lamd[t6] * (0.5 * ((xval[t2] + xval[t7])**2 - xval[t2] - xval[t7]) + self.opt_solution[t1] -  self.opt_solution[t5])            
        return t_coeff + [0] * (self.n_dim - len(t_coeff))
    
    def compute_qmat(self, lamd):
        quad_t_coeff = np.zeros((self.n_dim, self.n_dim))
        for j in range(n_stage - 1):
            for r in range(n_batch * pcast - 1):
                for k in range(n_machine_perstage):
                    for i in range(n_job):
                        for i1 in range(n_job):
                            if i1 != i:
                                t1 = i * n_job + i1 * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r
                                t2 = i1 * n_job + i * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r
                                quad_t_coeff[i + j * n_job, i + j * n_job] -= 0.5 * (lamd[t1] + lamd[t2])
                    for i in range(n_job-1):
                        for i1 in range(i+1,n_job):
                            t1 = i * n_job + i1 * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r
                            t2 = i1 * n_job + i * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r
                            quad_t_coeff[i + j * n_job, i1 + j * n_job] += 0.5 * (lamd[t1] + lamd[t2])
                            quad_t_coeff[i1 + j * n_job, i + j * n_job] += 0.5 * (lamd[t1] + lamd[t2])                            
        quad_t_coeff = quad_t_coeff.tolist()
        self.qmat = [[self.variable_names, quad_t_coeff[i]] for i in range(len(quad_t_coeff))]
        return self.qmat
    
    def solve(self):
        self.subproblem_cplex = cplex.Cplex()
        self.subproblem_cplex.objective.set_sense(self.subproblem_cplex.objective.sense.maximize)
        
        ub  = [cplex.infinity] * self.n_dim
        lb  = [0] * self.n_dim
        self.subproblem_cplex.variables.add(obj = self.obj_linear, ub = ub, lb = lb, names = self.variable_names)
        self.subproblem_cplex.objective.set_quadratic(self.qmat)
    
        rownames = ['c' + str(i)  for i in range(self.n_constraints)]

        self.subproblem_cplex.linear_constraints.add(lin_expr = self.constraints_rows, senses = self.senses, rhs = self.rhs, names = rownames)
        self.subproblem_cplex.solve()
        self.get_solvable_status()
        print ("Solution value:   ",  self.subproblem_cplex.solution.get_objective_value())
        self.get_optvalues()

    def write_model(self):
        self.subproblem_cplex.write('Subproblem_QP.lp')
        
    def get_optvalues(self):
        self.opt_solution = []
        for i, x in enumerate(self.subproblem_cplex.solution.get_values()):
            if i < self.n_dim:
                self.opt_solution.append(x)
        self.cost_function_iterations.append(self.subproblem_cplex.solution.get_objective_value())
                
    def get_solvable_status(self):
        status = self.subproblem_cplex.solution.get_status()
        if status == self.subproblem_cplex.solution.status.unbounded:
            print("Model is unbounded")
        if status == self.subproblem_cplex.solution.status.infeasible:
            print("Model is infeasible")
        if status == self.subproblem_cplex.solution.status.infeasible_or_unbounded:
            print("Model is infeasible or unbounded")
            
    def plot_results(self):
        plt.figure()
        x = np.linspace(0, len(self.cost_function_iterations), len(self.cost_function_iterations))
        plt.plot(x, self.cost_function_iterations)
        plt.xlabel('iteration times')
        plt.ylabel('cost function')
        plt.show()
        
    def output_results(self):
        with open('Subproblem_QP_result.txt', 'w') as f:
            for i in range(n_job * n_stage):
                f.write(str(self.opt_solution[i * n_job:(i + 1) * n_job]))
                f.write('\n')

In [30]:
print(len(processtime))
random.seed(0)
subproblem_qp = Subproblem_QP()
print(subproblem_qp.opt_solution)
print(subproblem_qp.compute_obj_wait_time())

135
[104, 88, 91]
[1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[-10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -20.0, -20.0, -20.0, -20.0, -20.0, -20.0, -20.0, -20.0, -20.0, -20.0, -20.0, -20.0, -20.0, -20.0, -20.0, -20.0, -20.0, -20.0, -20.0, -20.0, -20.0, -20.0, -20.0, -20.0, -2

In [31]:
print(subproblem_qp.compute_obj_casting())

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 110.0, 110.0, 110.0, 110.0, 110.0, 110.0, 110.0, 110.0, 110.0, 110.0, 110.0, 110.0, 110.0, 110.0, 110.0]


In [32]:
preceding_transfer_constraints, rhs_preceding_transfer, rhs_processtime, rhs_transittime = subproblem_qp.set_preceding_transfer_constraints()
print(preceding_transfer_constraints)
print(rhs_processtime)
print(len(rhs_processtime))
print(rhs_transittime)
print(len(rhs_transittime))
print(rhs_preceding_transfer)
print(len(rhs_preceding_transfer))

[[['t0', 't45'], [1, -1]], [['t1', 't46'], [1, -1]], [['t2', 't47'], [1, -1]], [['t3', 't48'], [1, -1]], [['t4', 't49'], [1, -1]], [['t5', 't50'], [1, -1]], [['t6', 't51'], [1, -1]], [['t7', 't52'], [1, -1]], [['t8', 't53'], [1, -1]], [['t9', 't54'], [1, -1]], [['t10', 't55'], [1, -1]], [['t11', 't56'], [1, -1]], [['t12', 't57'], [1, -1]], [['t13', 't58'], [1, -1]], [['t14', 't59'], [1, -1]], [['t15', 't60'], [1, -1]], [['t16', 't61'], [1, -1]], [['t17', 't62'], [1, -1]], [['t18', 't63'], [1, -1]], [['t19', 't64'], [1, -1]], [['t20', 't65'], [1, -1]], [['t21', 't66'], [1, -1]], [['t22', 't67'], [1, -1]], [['t23', 't68'], [1, -1]], [['t24', 't69'], [1, -1]], [['t25', 't70'], [1, -1]], [['t26', 't71'], [1, -1]], [['t27', 't72'], [1, -1]], [['t28', 't73'], [1, -1]], [['t29', 't74'], [1, -1]], [['t30', 't75'], [1, -1]], [['t31', 't76'], [1, -1]], [['t32', 't77'], [1, -1]], [['t33', 't78'], [1, -1]], [['t34', 't79'], [1, -1]], [['t35', 't80'], [1, -1]], [['t36', 't81'], [1, -1]], [['t37', '

In [33]:
process_continuous_constraints, adjacent_casts_constraints, rhs_continuous_processtime, rhs_adjacent_casts =  subproblem_qp.set_process_continuous_constraints()
print(process_continuous_constraints)
print(rhs_continuous_processtime)
print(adjacent_casts_constraints)
print(rhs_adjacent_casts)

[[['t90', 't91'], [1, -1]], [['t91', 't92'], [1, -1]], [['t93', 't94'], [1, -1]], [['t94', 't95'], [1, -1]], [['t96', 't97'], [1, -1]], [['t97', 't98'], [1, -1]], [['t99', 't100'], [1, -1]], [['t100', 't101'], [1, -1]], [['t102', 't103'], [1, -1]], [['t103', 't104'], [1, -1]], [['t105', 't106'], [1, -1]], [['t106', 't107'], [1, -1]], [['t108', 't109'], [1, -1]], [['t109', 't110'], [1, -1]], [['t111', 't112'], [1, -1]], [['t112', 't113'], [1, -1]], [['t114', 't115'], [1, -1]], [['t115', 't116'], [1, -1]], [['t117', 't118'], [1, -1]], [['t118', 't119'], [1, -1]], [['t120', 't121'], [1, -1]], [['t121', 't122'], [1, -1]], [['t123', 't124'], [1, -1]], [['t124', 't125'], [1, -1]], [['t126', 't127'], [1, -1]], [['t127', 't128'], [1, -1]], [['t129', 't130'], [1, -1]], [['t130', 't131'], [1, -1]], [['t132', 't133'], [1, -1]], [['t133', 't134'], [1, -1]]]
[39, 39, 38, 48, 43, 37, 41, 50, 50, 43, 40, 44, 47, 37, 41, 49, 44, 39, 45, 44, 40, 43, 45, 48, 41, 45, 40, 38, 49, 38]
[[['t92', 't93'], [1,

In [34]:
additional_constraints = subproblem_qp.set_additional_constraints()
print(additional_constraints)

[[['t90', 't135', 't150'], [1, -1, 1]], [['t93', 't136', 't151'], [1, -1, 1]], [['t96', 't137', 't152'], [1, -1, 1]], [['t99', 't138', 't153'], [1, -1, 1]], [['t102', 't139', 't154'], [1, -1, 1]], [['t105', 't140', 't155'], [1, -1, 1]], [['t108', 't141', 't156'], [1, -1, 1]], [['t111', 't142', 't157'], [1, -1, 1]], [['t114', 't143', 't158'], [1, -1, 1]], [['t117', 't144', 't159'], [1, -1, 1]], [['t120', 't145', 't160'], [1, -1, 1]], [['t123', 't146', 't161'], [1, -1, 1]], [['t126', 't147', 't162'], [1, -1, 1]], [['t129', 't148', 't163'], [1, -1, 1]], [['t132', 't149', 't164'], [1, -1, 1]]]


In [35]:
rows, rhs, n_per_constraints = subproblem_qp.set_constraints_byrows()
print(rows)
print(rhs)
print(n_per_constraints)

[[['t0', 't45'], [1, -1]], [['t1', 't46'], [1, -1]], [['t2', 't47'], [1, -1]], [['t3', 't48'], [1, -1]], [['t4', 't49'], [1, -1]], [['t5', 't50'], [1, -1]], [['t6', 't51'], [1, -1]], [['t7', 't52'], [1, -1]], [['t8', 't53'], [1, -1]], [['t9', 't54'], [1, -1]], [['t10', 't55'], [1, -1]], [['t11', 't56'], [1, -1]], [['t12', 't57'], [1, -1]], [['t13', 't58'], [1, -1]], [['t14', 't59'], [1, -1]], [['t15', 't60'], [1, -1]], [['t16', 't61'], [1, -1]], [['t17', 't62'], [1, -1]], [['t18', 't63'], [1, -1]], [['t19', 't64'], [1, -1]], [['t20', 't65'], [1, -1]], [['t21', 't66'], [1, -1]], [['t22', 't67'], [1, -1]], [['t23', 't68'], [1, -1]], [['t24', 't69'], [1, -1]], [['t25', 't70'], [1, -1]], [['t26', 't71'], [1, -1]], [['t27', 't72'], [1, -1]], [['t28', 't73'], [1, -1]], [['t29', 't74'], [1, -1]], [['t30', 't75'], [1, -1]], [['t31', 't76'], [1, -1]], [['t32', 't77'], [1, -1]], [['t33', 't78'], [1, -1]], [['t34', 't79'], [1, -1]], [['t35', 't80'], [1, -1]], [['t36', 't81'], [1, -1]], [['t37', '

In [39]:
duedate = subproblem_qp.compute_duedate()
print(duedate)
print(len(duedate))
print(processtime)

[104, 88, 91]
[104, 310, 520, 717, 932, 88, 292, 500, 720, 931, 91, 291, 506, 711, 908]
15
[49, 42, 48, 50, 42, 36, 40, 44, 43, 42, 50, 48, 49, 40, 43, 41, 45, 50, 50, 39, 44, 38, 40, 38, 48, 37, 45, 48, 40, 50, 44, 47, 48, 45, 50, 38, 40, 37, 47, 37, 50, 49, 46, 41, 43, 44, 37, 41, 42, 41, 45, 46, 50, 39, 44, 43, 43, 49, 44, 40, 36, 48, 50, 44, 50, 36, 37, 47, 49, 42, 47, 49, 48, 46, 46, 36, 45, 43, 49, 49, 41, 39, 47, 41, 47, 49, 37, 39, 50, 45, 39, 39, 48, 38, 48, 44, 43, 37, 37, 41, 50, 44, 50, 43, 37, 40, 44, 40, 47, 37, 44, 41, 49, 50, 44, 39, 48, 45, 44, 45, 40, 43, 37, 45, 48, 42, 41, 45, 39, 40, 38, 39, 49, 38, 36]


In [40]:
np.random.seed(0)
xval = [random.randint(0,1) for i in range((n_stage - 1) * n_job * n_job)]  
tval = [random.randint(0,1) for i in range(n_stage * n_job)] + 2 * pcast * n_machine['steelmaking'] * [0]
lamd = np.zeros((n_stage - 1) * n_job * n_job * n_job)
t_coeff = subproblem_qp.compute_F6_t(xval, lamd)
print("t_coeff:", len(t_coeff))
print(t_coeff)

t_coeff: 165
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [41]:
subproblem_qp.compute_qmat(lamd)
print(subproblem_qp.qmat)

[[['t0', 't1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10', 't11', 't12', 't13', 't14', 't15', 't16', 't17', 't18', 't19', 't20', 't21', 't22', 't23', 't24', 't25', 't26', 't27', 't28', 't29', 't30', 't31', 't32', 't33', 't34', 't35', 't36', 't37', 't38', 't39', 't40', 't41', 't42', 't43', 't44', 't45', 't46', 't47', 't48', 't49', 't50', 't51', 't52', 't53', 't54', 't55', 't56', 't57', 't58', 't59', 't60', 't61', 't62', 't63', 't64', 't65', 't66', 't67', 't68', 't69', 't70', 't71', 't72', 't73', 't74', 't75', 't76', 't77', 't78', 't79', 't80', 't81', 't82', 't83', 't84', 't85', 't86', 't87', 't88', 't89', 't90', 't91', 't92', 't93', 't94', 't95', 't96', 't97', 't98', 't99', 't100', 't101', 't102', 't103', 't104', 't105', 't106', 't107', 't108', 't109', 't110', 't111', 't112', 't113', 't114', 't115', 't116', 't117', 't118', 't119', 't120', 't121', 't122', 't123', 't124', 't125', 't126', 't127', 't128', 't129', 't130', 't131', 't132', 't133', 't134', 't135', 't136', 't137', 't1

In [42]:
subproblem_qp.compute_obj_linear(xval, lamd)
print(subproblem_qp.obj_linear)

[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -110.0, -110.0, -110.0, -110.0, -110.0,

In [43]:
subproblem_qp.solve()
subproblem_qp.write_model()
subproblem_qp.output_results()

CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
QP Presolve eliminated 1 rows and 1 columns.
Aggregator did 75 substitutions.
Reduced QP has 71 rows, 89 columns, and 157 nonzeros.
Presolve time = 0.00 sec. (0.09 ticks)

Iteration log . . .
Iteration:     1   Dual infeasibility =          1290.000000
Iteration:    45   Dual objective     =        -89560.000000
Solution value:    -92310.0


In [44]:
def compute_heuristic_xval(tval):
    xval = [0 for i in range((n_stage - 1) * n_job * n_job)]
    j = 0
    while (j < (n_stage - 1)):
        date = []
        for i in range(j * n_job, (j + 1) * n_job):
            date.append((tval[i],i))
        result = sorted(date)
        for i in range(n_job):
            job_index = result[i][1] - j * n_job
            t1 = i
            k = t1 % n_machine_perstage
            r = 0    
            while (t1 > n_machine_perstage - 1):
                r += 1
                t1 -= n_machine_perstage
            xval[job_index * n_job + j * n_job * n_job + k * n_batch * pcast + r] = 1
            t1 +=1
        j += 1
    return xval

In [45]:
x_init = compute_heuristic_xval(subproblem_qp.opt_solution)
with open('x_init.txt', 'w') as f:
    for i in range(n_job * (n_stage - 1)):
        f.write(str(x_init[i * n_job:(i + 1) * n_job]))
        f.write('\n')
        
with open('t_init.txt', 'w') as f:
    for i in range(n_job):
        f.write(str(subproblem_qp.opt_solution[i * n_job:(i + 1) * n_job]))
        f.write('\n')

In [46]:
class List_Schedule(Subproblem_QP):
    def __init__(self, xval):
        self.n_dim = n_job * n_stage + 2 * pcast * n_machine['steelmaking']
        self.variable_names = ['t' + str(i)  for i in range(self.n_dim)]
        self.xval = xval
        self.duedate = self.compute_duedate()
        self.constraints_rows, self.rhs, self.n_per_constraints = self.set_constraints_byrows()
        self.senses = self.set_senses()
        self.n_constraints = len(self.constraints_rows) 
        
    def compute_obj(self):
        t_coeff_w = self.compute_obj_wait_time()
        t_coeff_c = self.compute_obj_casting()
        t_coeff = [- t_coeff_w[i] - t_coeff_c[i] for i in range(self.n_dim)]
        return t_coeff
    
    def set_machine_capacity_constraints(self):
        machine_capacity_constraints = []
        rhs_machine_capacity_processtime = []
        for i in range(n_job):
            for i1 in range(n_job):
                if i != i1:
                    for j in range(n_stage - 1):
                        t1 = i + j * n_job
                        t2 = i1 + j * n_job
                        for r in range(n_batch * pcast - 1):
                            for k in range(n_machine_perstage):
                                t3 = i * n_job + j * n_job * n_job + k * n_batch * pcast + r
                                t4 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r + 1 
                                t5 = i * n_job + i1 * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r
                                if self.xval[t3] == 1 and self.xval[t4] == 1:
                                    machine_capacity_constraints.append([[self.variable_names[t2], self.variable_names[t1]],[-self.xval[t3] * self.xval[t4], self.xval[t3] * self.xval[t4]]])
                                    rhs_machine_capacity_processtime.append(-self.xval[t3] * self.xval[t4] * processtime[t1]) 
        return machine_capacity_constraints, rhs_machine_capacity_processtime
    
    def set_constraints_byrows(self):
        machine_capacity_constraints, rhs_machine_capacity_processtime = self.set_machine_capacity_constraints()
        preceding_transfer_constraints, rhs_preceding_transfer, _, _ = self.set_preceding_transfer_constraints()
        process_continuous_constraints, adjacent_casts_constraints, rhs_continuous_processtime, rhs_adjacent_casts = self.set_process_continuous_constraints()
        additional_constraints = subproblem_qp.set_additional_constraints()
    
        rows = machine_capacity_constraints + preceding_transfer_constraints + process_continuous_constraints + adjacent_casts_constraints + additional_constraints
        
        n_per_constraints = [len(machine_capacity_constraints), len(preceding_transfer_constraints), len(process_continuous_constraints), len(adjacent_casts_constraints), len(additional_constraints)]
        
        rhs = rhs_machine_capacity_processtime + rhs_preceding_transfer + rhs_continuous_processtime + rhs_adjacent_casts + self.duedate
        return rows, rhs, n_per_constraints
        
    def set_senses(self):
        s, senses = 'LLELE', ''
        for i in range(len(self.n_per_constraints)):
            senses += s[i] * self.n_per_constraints[i]
        return senses 
    
    def solve(self):
        self.List_Schedule = cplex.Cplex()
        self.List_Schedule.objective.set_sense(self.List_Schedule.objective.sense.maximize)
        
        ub = [cplex.infinity] * self.n_dim
        #ub = [1000] * self.n_dim
        self.List_Schedule.variables.add(obj = self.compute_obj(), ub = ub, names = self.variable_names)
    
        rownames = ['c' + str(i)  for i in range(self.n_constraints)]

        self.List_Schedule.linear_constraints.add(lin_expr = self.constraints_rows, senses = self.senses, rhs = self.rhs, names = rownames)
        self.List_Schedule.solve()
        self.get_solvable_status()
        self.get_optvalues()
    
    def write_model(self):
        self.List_Schedule.write('Subproblem_QP.lp')
        
    def get_optvalues(self):
        self.opt_solution = []
        for i, x in enumerate(self.List_Schedule.solution.get_values()):
            if i < self.n_dim:
                self.opt_solution.append(x)
                
    def get_solvable_status(self):
        status = self.List_Schedule.solution.get_status()
        if status == self.List_Schedule.solution.status.unbounded:
            print("Model is unbounded")
        if status == self.List_Schedule.solution.status.infeasible:
            print("Model is infeasible")
        if status == self.List_Schedule.solution.status.infeasible_or_unbounded:
            print("Model is infeasible or unbounded")
            
    def write_model(self):
        self.List_Schedule.write('List_Schedule.lp')

In [49]:
random.seed(0)
list_schedule = List_Schedule(x_init)
list_schedule.solve()
list_schedule.write_model()
#list_schedule.solve()
with open('list_schedule.txt', 'w') as f:
    for i in range(n_job):
        f.write(str(list_schedule.opt_solution[i * n_job:(i + 1) * n_job]))
        f.write('\n')

[104, 88, 91]
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
Aggregator did 33 substitutions.
Reduced LP has 198 rows, 132 columns, and 411 nonzeros.
Presolve time = 0.00 sec. (0.13 ticks)

Iteration log . . .
Iteration:     1   Dual infeasibility =          1330.000000
Iteration:    62   Dual infeasibility =           250.000000
Iteration:    88   Dual objective     =        -89510.000000


In [50]:
list_schedule.write_model()
list_schedule.constraints_rows[0]

[['t60', 't45'], [-1, 1]]

In [ ]:
class Dual_Problem():
    def __init__(self):
        self.lamd = np.array([1 for i in range((n_stage - 1) * n_job * n_job * n_job)])
        self.m, self.r, self.alpha = 2, 0.01, 1.0
        self.step_init, self.step, self.iteration_time = 1.0, 0.0, 0
        self.subgradients_init = np.array([1 for i in range((n_stage - 1) * n_job * n_job * n_job)])
        self.subgradients = np.array([0 for i in range((n_stage - 1) * n_job * n_job * n_job)])
        
    def compute_subgradients(self, xval, tval):
        if self.iteration_time == 0:
            self.subgradients_init = np.array([1 for i in range((n_stage - 1) * n_job * n_job * n_job)])
        else:
            for i in range(n_job):
                for i1 in range(n_job):
                    if i != i1:
                        for j in range(n_stage - 1):
                            t1 = i + j * n_job
                            t2 = i1 + j * n_job
                            for r in range(n_batch * pcast - 1):
                                for k in range(n_machine_perstage):
                                    t3 = i * n_job + j * n_job * n_job + k * n_batch * pcast + r
                                    t4 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r + 1 
                                    t5 = i * n_job + i1 * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r
                                    self.subgradients[t5] = - xval[t3] * xval[t4] * (tval[t2] - tval[t1] - processtime[t1])
        return self.subgradients
    
    def compute_relaxed_problem_obj(self):
        LR_fun_value = 0
        t_coeff_w = subproblem_qp.compute_obj_wait_time()
        t_coeff_c = subproblem_qp.compute_obj_casting()
        t_coeff = [t_coeff_w[i] + t_coeff_c[i] for i in range(len(t_coeff_w))]
        for i in range(len(t_coeff)):
            LR_fun_value += t_coeff[i] * subproblem_qp.opt_solution[i] 

        for i in range(n_job):
            for i1 in range(n_job): 
                if i != i1:
                    for j in range(n_stage - 1):
                        t1 = i + j * n_job
                        t2 = i1 + j * n_job
                        for k in range(n_machine_perstage):
                            for r in range(n_batch * pcast - 1):
                                t3 = i * n_job + j * n_job * n_job + k * n_batch * pcast + r
                                t4 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r + 1 
                                t5 = i * n_job + i1 * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r
                                #print(t3, t4, subproblem_lp.opt_solution[t4], subproblem_lp.opt_solution[t3], subproblem_qp.opt_solution[t2], subproblem_qp.opt_solution[t1], processtime[t1])
                                #value.append(-self.lamd[t5] * subproblem_lp.opt_solution[t3] * subproblem_lp.opt_solution[t4] * (subproblem_qp.opt_solution[t2] - subproblem_qp.opt_solution[t1] - processtime[t1]))
                                LR_fun_value -= self.lamd[t5] * subproblem_lp.opt_solution[t3] * subproblem_lp.opt_solution[t4] * (subproblem_qp.opt_solution[t2] - subproblem_qp.opt_solution[t1] - processtime[t1])
        return LR_fun_value
    
    def compute_stepsize(self):
        if self.iteration_time == 0:
            self.step = self.step_init
        else:
            p = 1 - 1/(self.iteration_time ** self.r)
            self.alpha = self.alpha * (1 - 1/(self.m * self.iteration_time ** p))
            if np.linalg.norm(self.subgradients) != 0:
                self.step = self.alpha * self.step_init * np.linalg.norm(self.subgradients_init) / np.linalg.norm(self.subgradients)
            else:
                self.step = 0
                print("subgradients = 0")
        return self.step
    
    def update_lamd(self):
        if self.iteration_time == 0:
            self.lamd = self.lamd + self.step_init * self.subgradients_init
        else:
            self.lamd = self.lamd + self.step * self.subgradients
        self.lamd = np.maximum(self.lamd, 0)
        self.iteration_time += 1
        return self.lamd

In [ ]:
dual_problem = Dual_Problem()
dual_problem = Dual_Problem.compute_subgradients(xval, tval)
print(dual_problem.subgradients)

In [ ]:
LR_fun_value = subgradients_algorithm.compute_relaxed_problem_obj()
print(LR_fun_value)

In [ ]:
stepsize = dual_problem.compute_stepsize()
print(dual_problem.step)

In [ ]:
print(dual_problem.update_lamd())